<a href="https://colab.research.google.com/github/VinuthaTC/vinu/blob/main/Tweet_NFT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import  PorterStemmer
import string
special = string.punctuation

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding,LSTM, Dense,Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot

In [ ]:
from sklearn.impute import SimpleImputer

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_excel("/content/Tweet_NFT.xlsx",sheet_name='Sheet1')

In [ ]:
df.head()

,id,tweet_text,tweet_created_at,tweet_intent
0,1212762.0,@crypto_brody @eCoLoGy1990 @MoonrunnersNFT @It...,2022-08-06T16:56:36.000Z,Community
1,1212763.0,Need Sick Character artâ_x009d_“#art #artist #...,2022-08-06T16:56:36.000Z,Giveaway
2,1212765.0,@The_Hulk_NFT @INagotchiNFT @Tesla @killabears...,2022-08-06T16:56:35.000Z,Appreciation
3,1212766.0,@CryptoBatzNFT @DarekBTW The first project in ...,2022-08-06T16:56:35.000Z,Community
4,1212767.0,@sashadysonn The first project in crypto with ...,2022-08-06T16:56:34.000Z,Community


In [ ]:
df.shape

(127453, 4)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 127453 entries, 0 to 127452
Data columns (total 4 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   id                127453 non-null  float64
 1   tweet_text        127453 non-null  object 
 2   tweet_created_at  127453 non-null  object 
 3   tweet_intent      96364 non-null   object 
dtypes: float64(1), object(3)
memory usage: 3.9+ MB


In [ ]:
df.describe()

,id
count,1.274530e+05
mean,1.297300e+06
std,6.301816e+04
min,1.212762e+06
25%,1.244749e+06
50%,1.276623e+06
75%,1.346851e+06
max,2.309537e+06


In [ ]:
df.isnull().sum()

id                      0
tweet_text              0
tweet_created_at        0
tweet_intent        31089
dtype: int64

In [ ]:
def missing_data(data):
    total = data.isnull().sum()
    percent = (data.isnull().sum()/data.isnull().count()*100)
    tt = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
    types = []
    for col in data.columns:
        dtype = str(data[col].dtype)
        types.append(dtype)
    tt['Types'] = types
    return(np.transpose(tt))

In [ ]:
missing_data(df)

,id,tweet_text,tweet_created_at,tweet_intent
Total,0,0,0,31089
Percent,0.0,0.0,0.0,24.392521
Types,float64,object,object,object


In [ ]:
def unique_values(data):
    total = data.count()
    tt = pd.DataFrame(total)
    tt.columns = ['Total']
    uniques = []
    for col in data.columns:
        unique = data[col].nunique()
        uniques.append(unique)
    tt['Uniques'] = uniques
    return(np.transpose(tt))

In [ ]:
unique_values(df)

,id,tweet_text,tweet_created_at,tweet_intent
Total,127453,127453,127453,96364
Uniques,127453,114494,50394,9


In [ ]:
def most_frequent_values(data):
    total = data.count()
    tt = pd.DataFrame(total)
    tt.columns = ['Total']
    items = []
    vals = []
    for col in data.columns:
        itm = data[col].value_counts().index[0]
        val = data[col].value_counts().values[0]
        items.append(itm)
        vals.append(val)
    tt['Most frequent item'] = items
    tt['Frequence'] = vals
    tt['Percent from total'] = np.round(vals / total * 100, 3)
    return(np.transpose(tt))

In [ ]:
most_frequent_values(df)

,id,tweet_text,tweet_created_at,tweet_intent
Total,127453,127453,127453,96364
Most frequent item,1212762.0,@Codex314159265 Feature it on @Globalnft07\nWe...,2022-08-11T10:57:24.000Z,Community
Frequence,1,381,24,41510
Percent from total,0.001,0.299,0.019,43.076


In [ ]:
feature=df['tweet_text'].copy()
target=df['tweet_intent'].copy()
print(feature.shape)
print(target.shape)

(127453,)
(127453,)


In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
ps=PorterStemmer()
word_list=[]

In [ ]:
for i in range(len(feature)):
  review=re.sub('^a-zA-Z',' ',feature[i])
  review=re.sub(r'[^\w\s]', '', review) #remove puntuation
  review=re.sub(r'[0-9]+', '', review) #remove Number
  review=review.lower()
  review=review.split()
  review=[ps.stem(word) for word in review if not word in stopwords.words('english')]
  review=' '.join(review)
  word_list.append(review)

In [ ]:
word_list[1:5]

print(len(word_list))

127453


In [ ]:
vc_size=127453 #define number of vocab size


word_list_onhot=[one_hot(word,vc_size) for word in word_list]
word_list_onhot[1]

[78185,
 101152,
 65882,
 5710,
 64108,
 96592,
 46098,
 46098,
 25635,
 74182,
 74182,
 91409,
 101059,
 108398,
 92671,
 72653,
 80204,
 72100,
 74182,
 21122,
 33363,
 59852,
 3033,
 86557,
 50082,
 106999,
 18238,
 15353]

In [ ]:
sentence_len=[]
for i in range(len(word_list_onhot)):
  x=len(word_list_onhot[i])
  sentence_len.append(x)

In [ ]:
# x='dfsdf'
len(word_list)

127453

In [ ]:

# Creating the Bag of Words model
from sklearn.feature_extraction.text import CountVectorizer
 
# To extract max 1500 feature.
# "max_features" is attribute to
# experiment with to get better results
cv = CountVectorizer(max_features = 1500)
 
# X contains corpus (dependent variable)
X = cv.fit_transform(word_list).toarray()
 
# y contains answers if review
# is positive or negative
y = df.iloc[:, 1].values

Padding

In [ ]:
print(type(sentence_len))
sen_len=max(sentence_len)
print('Max length of a line is :',sen_len)

<class 'list'>
Max length of a line is : 237


In [ ]:
word_embeding=pad_sequences(word_list_onhot,padding='pre',maxlen=sen_len)
word_embeding.shape

(127453, 237)

Create Model

In [ ]:
model=Sequential()
model.add(Embedding(vc_size,25,input_length=sen_len))
model.add(Dropout(0.1))
model.add(LSTM(100))
model.add(Dropout(0.1))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 237, 25)           3186325   
                                                                 
 dropout (Dropout)           (None, 237, 25)           0         
                                                                 
 lstm (LSTM)                 (None, 100)               50400     
                                                                 
 dropout_1 (Dropout)         (None, 100)               0         
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 3,236,826
Trainable params: 3,236,826
Non-trainable params: 0
_________________________________________________________________


In [ ]:
def vectorization(table):
    #CountVectorizer will convert a collection of text documents to a matrix of token counts
    #Produces a sparse representation of the counts 
    #Initialize
    vector = CountVectorizer()
    #We fit and transform the vector created
    frequency_matrix = vector.fit_transform(table.tweet)
    #Sum all the frequencies for each word
    sum_frequencies = np.sum(frequency_matrix, axis=0)
    #Now we use squeeze to remove single-dimensional entries from the shape of an array that we got from applying np.asarray to
    #the sum of frequencies.
    frequency = np.squeeze(np.asarray(sum_frequencies))
    #Now we get into a dataframe all the frequencies and the words that they correspond to
    frequency_df = pd.DataFrame([frequency], columns=vector.get_feature_names()).transpose()
    return frequency_df

Split train and test data

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(word_embeding, target, test_size=0.3, random_state=42)

In [ ]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(89217, 237) (38236, 237) (89217,) (38236,)


In [ ]:
model_hist=model.fit(word_embeding,target, validation_data=(X_test,y_test),epochs=10,batch_size=32,)

ValueError: ignored

In [ ]:
from sklearn.metrics import confusion_matrix,accuracy_score

In [ ]:
# pred=model.predict_classes(X_test)
pred = model.predict(X_test)
pred = np.round(pred).astype(int)

In [ ]:
confusion_matrix(y_test,pred)

TypeError: ignored

In [ ]:
accuracy_score(y_test,pred)*100

TypeError: ignored

My model provide around 97% accuracy. Thats prety good

In [ ]:
# save model
model.save('NLP_for_tweet_intent')

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
model=load_model("NLP_for_tweet_intent")

Prepare Test dataset

In [ ]:
test.head()

NameError: ignored

In [ ]:
test_feature=test['text']

In [ ]:
test_word_list=[]
for i in range(len(test_feature)):
  review=re.sub('^a-zA-Z',' ',test_feature[i])
  review=re.sub(r'[^\w\s]', '', review) #remove puntuation
  review=re.sub(r'[0-9]+', '', review) #remove Number
  review=review.lower()
  review=review.split()
  review=[ps.stem(word) for word in review if not word in stopwords.words('english')]
  review=' '.join(review)
  test_word_list.append(review)

In [ ]:
test_word_list[0:5]

In [ ]:
test_word_list_onhot=[one_hot(word,vc_size) for word in test_word_list]

In [ ]:
sentence_len=[]
for i in range(len(test_word_list_onhot)):
  x=len(word_list_onhot[i])
  sentence_len.append(x)
print(type(sentence_len))
sen_len=max(sentence_len)
print('Max length of a line is :',sen_len)

In [ ]:
test_word_embeding=pad_sequences(test_word_list_onhot,padding='pre',maxlen=sen_len)
test_word_embeding.shape